# For colab environment

In [1]:
# !pip install nltk transformers==4.35.0 torch==2.6.0 torchvision==0.21.0 datasets accelerate==0.24.0 huggingface==0.0.1 datasets==2.14.7

In [2]:
import torch 
print(torch.cuda.is_available())
print(torch.__version__)

True
2.6.0+cu124


In [3]:
# !git clone https://github.com/BernardMoy/NLP-PCL-Classification.git

In [4]:
# %cd NLP-PCL-Classification/

In [5]:
!nvidia-smi

Sun Mar  1 12:40:30 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA TITAN Xp                Off |   00000000:02:00.0  On |                  N/A |
| 23%   39C    P8             19W /  250W |     404MiB /  12288MiB |     32%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Load train and validation data set

In [6]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
from collections import Counter

df = pd.read_csv('data/dontpatronizeme_pcl.tsv', sep='\t')

# Remove rows with NA labels 
df = df.dropna() 

# Add a bool_labels column for binary classification
df["bool_labels"] = df["label"] > 1   # is PCL if >1

# train val split 
train_labels = pd.read_csv('data/train_semeval_parids-labels.csv')["par_id"]
val_labels = pd.read_csv('data/dev_semeval_parids-labels.csv')["par_id"]
df_train = df[df["par_id"].isin(train_labels)].reset_index() 
df_val = df[df["par_id"].isin(val_labels)].reset_index() 


# Text Cleaning

In [7]:
# Remove special characters
SPECIAL_CHARACTERS = ['&amp;', '&lt;', '&gt;', '<h>', '\n', '\t']
for char in SPECIAL_CHARACTERS:
    df_train["text"] = df_train["text"].str.replace(char, "")
    df_val["text"] = df_val["text"].str.replace(char, "")


print(df_train["text"].iloc[55])


# Replace numbers with 0
df_train["text"] = df_train["text"].str.replace(r"\d+", "0", regex=True)
df_val["text"] = df_val["text"].str.replace(r"\d+", "0", regex=True)

print(df_train["text"].iloc[3])

People who are homeless , those who were once homeless , those working with the homeless and concerned New Zealanders are being asked to share their experiences and solutions to this growing issue with the Cross-Party Homelessness Inquiry . More
Council customers only signs would be displayed . Two of the spaces would be reserved for disabled persons and there would be five P0 spaces and eight P0 ones .


# Coreference resolution

In [8]:
from fastcoref import FCoref

# define the model once
model = FCoref(device='cuda:0' if torch.cuda.is_available() else 'cpu')


def coreference_resolution(model, text):
    # Batch coreference resolution for all texts 
    preds = model.predict(
        texts = text
    )

    # Iterate each row of the list to substitute the pronouns / references with entity names 
    result = [] 
    for i in range(len(text)): 
        sent = text[i] 
        clusters = preds[i].get_clusters(as_strings = False) 

        # create mappings from each pronoun indices -> entities TEXT
        d = {}
        for cluster in clusters:
            entity = cluster[0]    # IMPORTANT - The first entity is assumed to be the main entity here. Use POS tagging to further improve this. 
            refs = cluster[1::]

            for ref in refs: 
                d[ref] = sent[entity[0]:entity[1]]

        # for each pronoun index (key), replace by their entity text (value) 
        sorted_keys = sorted(d.keys(), reverse = True) 
        for key in sorted_keys: 
            start, end = key 
            sent = sent[:start] + d[key] + sent[end:]

        result.append(sent) 

    return result 

   

test = coreference_resolution(model, ['We are so happy to see you using our coref package . This package is very fast !', 
                                     'He said the CEO of Apple was happy. Tim Cook later confirmed it .', 
                                     "Dr. Lester Keith , doctor and professor of business administration , and others are checking with local transportation groups to see if they can bring those in need of a meal to the college for the 4 p.m. dinner . We will also be contacting local soup kitchens as a pickup location and will work with them to transport any leftovers to them so there is no wasted food , Dr. Keith said ."])
print(test[0]) 
print(test[1]) 
print(test[2])


/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
03/01/2026 12:40:49 - INFO - 	 missing_keys: []
03/01/2026 12:40:49 - INFO - 	 unexpected_keys: []
03/01/2026 12:40:49 - INFO - 	 mismatched_keys: []
03/01/2026 12:40:49

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

03/01/2026 12:40:49 - INFO - 	 ***** Running Inference on 3 texts *****


Inference:   0%|          | 0/3 [00:00<?, ?it/s]

We are so happy to see you using We coref package . our coref package is very fast !
He said the CEO of Apple was happy. the CEO of Apple later confirmed it .
Dr. Lester Keith , doctor and professor of business administration , and others are checking with local transportation groups to see if local transportation groups can bring those in need of a meal to the college for the 4 p.m. dinner . Dr. Lester Keith , doctor and professor of business administration , and others will also be contacting local soup kitchens as a pickup location and will work with local soup kitchens to transport any leftovers to local soup kitchens so there is no wasted food , Dr. Lester Keith , doctor and professor of business administration said .


In [9]:
train_coref = coreference_resolution(model, df_train["text"].tolist())
df_train["text_cr"] = pd.Series(train_coref) 

val_coref = coreference_resolution(model, df_val["text"].tolist())
df_val["text_cr"] = pd.Series(val_coref) 

03/01/2026 12:40:49 - INFO - 	 Tokenize 8375 inputs...


Map:   0%|          | 0/8375 [00:00<?, ? examples/s]

03/01/2026 12:41:02 - INFO - 	 ***** Running Inference on 8375 texts *****


Inference:   0%|          | 0/8375 [00:00<?, ?it/s]

03/01/2026 12:41:16 - INFO - 	 Tokenize 2093 inputs...


Map:   0%|          | 0/2093 [00:00<?, ? examples/s]

03/01/2026 12:41:20 - INFO - 	 ***** Running Inference on 2093 texts *****


Inference:   0%|          | 0/2093 [00:00<?, ?it/s]

In [10]:
print(df_train["text"].iloc[23])
print(df_train["text_cr"].iloc[23])

print(df_val["text"].iloc[23])
print(df_val["text_cr"].iloc[23])

" The regional brands so far lag behind the global and big international Chinese handset players in 0G and they have looked vulnerable to failing to jump the generation successfully and lose their place . "
" The regional brands so far lag behind the global and big international Chinese handset players in 0G and The regional brands have looked vulnerable to failing to jump the generation successfully and lose The regional brands place . "
BUSINESSMAN Norberto Quisumbing Jr . of the Norkis Group of Companies has a challenge for families who can spare some of what they have : why not adopt poor families and help them break the cycle of poverty ?
BUSINESSMAN Norberto Quisumbing Jr . of the Norkis Group of Companies has a challenge for families who can spare some of what families who can spare some of what they have have : why not adopt poor families and help poor families break the cycle of poverty ?


# Add contextual information to the text tokens

In [11]:
def add_info(df): 
    # Append the keyword and country code to the text, and separate them with additional separator tokens
    # Remove dashes in the keyword to match the format in the texts 
    return df["keyword"].str.replace('-', " ") + "<SEP>" + df["country_code"] + "<SEP>" + df["text_cr"]

# Tokenization

In [12]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoConfig, Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained("roberta-base") 

# define special tokens for separating text 
special_tokens = {"additional_special_tokens": ["<SEP>"]}
tokenizer.add_special_tokens(special_tokens) 

# Create text with contextual information 
def tokenize(df): 
    text_with_context = add_info(df) 

    encoding = tokenizer(
        text_with_context.tolist(), 
        padding="max_length",   # Add padding to shorter sentences 
        max_length=256,
        truncation = True, 
        return_attention_mask = True 
    )

    return encoding

# Convert to pyTorch dataset

In [13]:
import torch 
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset

def to_dataset(df): 
    # Obtain tokens (input_ids, attention_mask) from the dataset 
    encoding = tokenize(df) 

    # Return huggingface dataset 
    return Dataset.from_dict({
        "input_ids": encoding["input_ids"], 
        "attention_mask": encoding["attention_mask"], 
        "label": df["bool_labels"].values 
    })

In [14]:
train_dataset = to_dataset(df_train)
val_dataset = to_dataset(df_val) 

# set to torch format 
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Training 

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate metrics 
    accuracy = accuracy_score(labels, predictions) 
    precision = precision_score(labels, predictions) 
    recall = recall_score(labels, predictions) 
    f1 = f1_score(labels, predictions) 

    return {
        "accuracy": accuracy, 
        "precision": precision, 
        "recall": recall, 
        "f1": f1 
    }


In [16]:
# Load roberta sequence classification model 
config = AutoConfig.from_pretrained("roberta-base", num_labels=2)  # Binary classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config = config)
model.resize_token_embeddings(len(tokenizer)) 

# Core hyperparameters 
BATCH_SIZE = 32
N_EPOCHS = 5 

# Set up training arguments 
training_args = TrainingArguments(
    fp16=True, 
    num_train_epochs=N_EPOCHS, 
    learning_rate=2e-5, 
    weight_decay=0.01,
    warmup_steps=500, 
    save_strategy="epoch",  # low disk space 
    load_best_model_at_end=True, 
    metric_for_best_model='f1',
    logging_steps=50,
    output_dir="./checkpoints/only_cr", 
    evaluation_strategy="epoch", 
    per_device_eval_batch_size=BATCH_SIZE, 
    per_device_train_batch_size=BATCH_SIZE, 
)

# Set up trainer 
trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    compute_metrics=compute_metrics
)


/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [17]:
trainer.train() 

{'loss': 0.7221, 'learning_rate': 1.9600000000000003e-06, 'epoch': 0.19}
{'loss': 0.4028, 'learning_rate': 3.96e-06, 'epoch': 0.38}
{'loss': 0.2924, 'learning_rate': 5.9600000000000005e-06, 'epoch': 0.57}
{'loss': 0.2408, 'learning_rate': 7.960000000000002e-06, 'epoch': 0.76}
{'loss': 0.2424, 'learning_rate': 9.960000000000001e-06, 'epoch': 0.95}
{'eval_loss': 0.21669618785381317, 'eval_accuracy': 0.9116101290014333, 'eval_precision': 0.5714285714285714, 'eval_recall': 0.2814070351758794, 'eval_f1': 0.3771043771043771, 'eval_runtime': 17.2249, 'eval_samples_per_second': 121.51, 'eval_steps_per_second': 3.832, 'epoch': 1.0}
{'loss': 0.2063, 'learning_rate': 1.196e-05, 'epoch': 1.15}
{'loss': 0.2277, 'learning_rate': 1.396e-05, 'epoch': 1.34}
{'loss': 0.2226, 'learning_rate': 1.5960000000000003e-05, 'epoch': 1.53}
{'loss': 0.2024, 'learning_rate': 1.796e-05, 'epoch': 1.72}
{'loss': 0.1953, 'learning_rate': 1.9960000000000002e-05, 'epoch': 1.91}
{'eval_loss': 0.25621864199638367, 'eval_ac

TrainOutput(global_step=1310, training_loss=0.17377396468215317, metrics={'train_runtime': 1141.5179, 'train_samples_per_second': 36.684, 'train_steps_per_second': 1.148, 'train_loss': 0.17377396468215317, 'epoch': 5.0})

In [18]:
trainer.evaluate()

{'eval_loss': 0.2649511694908142, 'eval_accuracy': 0.9235547061634019, 'eval_precision': 0.6167664670658682, 'eval_recall': 0.5175879396984925, 'eval_f1': 0.5628415300546448, 'eval_runtime': 16.5869, 'eval_samples_per_second': 126.184, 'eval_steps_per_second': 3.979, 'epoch': 5.0}


{'eval_loss': 0.2649511694908142,
 'eval_accuracy': 0.9235547061634019,
 'eval_precision': 0.6167664670658682,
 'eval_recall': 0.5175879396984925,
 'eval_f1': 0.5628415300546448,
 'eval_runtime': 16.5869,
 'eval_samples_per_second': 126.184,
 'eval_steps_per_second': 3.979,
 'epoch': 5.0}

# Save model

In [ ]:
trainer.save_model('models/model_only_cr')

: 